In [18]:
import os
from dotenv import load_dotenv
from pyathena import connect
from pyathena.pandas.util import as_pandas
from pyathena.arrow.cursor import ArrowCursor
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

load_dotenv()
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
region_name = os.getenv("AWS_REGION")
s3_staging_dir = os.getenv("S3_STAGING_DIR")

conn = connect(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    s3_staging_dir=s3_staging_dir,
    region_name=region_name)

In [6]:
def executa_query(cursor, query, success_message, error_message):
    try:
        cursor.execute(query)
        print(success_message)
    except Exception as e:
        print(f"{error_message}: {e}")

def read_query(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def main():
    try:
        with conn.cursor() as cursor:
            drop_table_queries = [
                "DROP TABLE IF EXISTS prevencao_a_fraude.qtd_falta;",
                "DROP TABLE IF EXISTS prevencao_a_fraude.data_inventario;",
                # "drop table if exists prevencao_a_fraude.vendas_cambraia;"
            ]
            for query in drop_table_queries:
                executa_query(cursor, query, "Tabela excluída com sucesso.", "Erro ao excluir tabela")

            query_dt_inv = read_query('kf_data_inv.sql')
            query_qtd_falta = read_query('kf_qtd_falta.sql')
            # query_venda = read_query('vendas.sql')
            query_cancelamento = read_query('cancelamento_xSA.sql')

            executa_query(cursor, query_dt_inv, "Tabela data_inventario criada com sucesso.", "Erro ao criar tabela data_inventario")
            executa_query(cursor, query_qtd_falta, "Tabela qtd_falta criada com sucesso.", "Erro ao criar tabela qtd_falta")
            

            # Executar consulta final
            df_canc = as_pandas(cursor.execute(query_cancelamento))
            print("Query de cancelamentos_xSA finalizada!")
            print("Número de linhas:\t", len(df_canc))

            # executa_query(cursor, query_venda, "Tabela vendas_cambraia criada com sucesso.", "Erro ao criar tabela vendas_cambraia")

            # Converter DataFrame pandas para uma tabela PyArrow e tranforma em parquet
            pq.write_table(pa.Table.from_pandas(df_canc), 'cancelamentos.parquet')

    except Exception as e:
        print(f"Erro geral: {e}")
    finally:
        conn.close()
    return df_canc

df_canc = main()

Tabela excluída com sucesso.
Tabela excluída com sucesso.
Tabela data_inventario criada com sucesso.
Tabela qtd_falta criada com sucesso.
Query de cancelamentos_xSA finalizada!
Número de linhas:	 3156


In [84]:
df = pd.read_parquet("cancelamentos.parquet")
display(df.loc[df['FILIAL'] == '135'])

,DATA_INVENTARIO,DATA_CPM,HORA_AB,HORA_FC,CHV_FLIAL_PRD,SKU,FILIAL,ECF,CAIXA,NOME_OP,...,QUANT,VALOR,CANC,AUTORIZADOR,NOME_AUT,FUNCAO,ITEM_CANC,MOTIVO,TIPO_MOV,QTD_FALTA
648,2024-08-08,2024-06-25,17:32:46,17:36:29,135|101681,1,135,103,135466,PEDRO HENRIQUE SANTOS SARAIVA,...,3,29.97,O,25000,POLICENA TERCEIRA BARBOZA DE ARAUJO,FARMACEUTICO(A) GERENTE RT,N,,SA,1
649,2024-08-08,2024-06-26,10:19:14,10:23:48,135|149692,1,135,100,80378,LUZINETE ROSA DE PONTES,...,1,255.42,O,123186,MARCOS MAX LEITE DA SILVA,OPERADOR DE LOJA,N,,SA,2
650,2024-08-08,2024-06-27,13:23:46,13:24:33,135|102322,1,135,103,76152,DAVI ADONAY GOMES DE SENA,...,1,16.70,O,123186,MARCOS MAX LEITE DA SILVA,OPERADOR DE LOJA,N,,SA,1
651,2024-08-08,2024-06-27,21:42:20,21:43:37,135|150564,1,135,100,135466,PEDRO HENRIQUE SANTOS SARAIVA,...,1,12.31,O,123186,MARCOS MAX LEITE DA SILVA,OPERADOR DE LOJA,S,,SA,1
652,2024-08-08,2024-06-28,22:45:03,22:46:39,135|150882,1,135,100,76152,DAVI ADONAY GOMES DE SENA,...,1,39.90,O,123186,MARCOS MAX LEITE DA SILVA,OPERADOR DE LOJA,N,,SA,1
653,2024-08-08,2024-06-29,21:24:41,21:26:44,135|103197,1,135,103,135466,PEDRO HENRIQUE SANTOS SARAIVA,...,9,17.91,O,68421,GENILSON AUGUSTO DA SILVA,OPERADOR DE LOJA,N,,SA,1
654,2024-08-08,2024-07-02,08:52:26,08:53:26,135|103862,1,135,103,80378,LUZINETE ROSA DE PONTES,...,1,31.00,O,34979,EDNALDA ALVES DE FRANCA MARINHO,FARMACEUTICO JR,N,,SA,1
655,2024-08-08,2024-07-02,19:35:00,19:35:49,135|151975,1,135,100,135466,PEDRO HENRIQUE SANTOS SARAIVA,...,1,57.37,O,123186,MARCOS MAX LEITE DA SILVA,OPERADOR DE LOJA,N,,SA,2
656,2024-08-08,2024-07-03,20:51:15,20:52:08,135|104680,1,135,103,135466,PEDRO HENRIQUE SANTOS SARAIVA,...,3,28.17,O,123186,MARCOS MAX LEITE DA SILVA,OPERADOR DE LOJA,N,,SA,5
657,2024-08-08,2024-07-03,21:04:22,21:04:53,135|104685,1,135,103,135466,PEDRO HENRIQUE SANTOS SARAIVA,...,1,104.35,O,123186,MARCOS MAX LEITE DA SILVA,OPERADOR DE LOJA,N,,SA,1


In [81]:
cancelados_op = df_canc[['CAIXA', 'NOME_OP', 'FILIAL', 'NUMERO_COO']].drop_duplicates()
cancelados_op = cancelados_op.groupby(['CAIXA', 'NOME_OP', 'FILIAL']).count()['NUMERO_COO'].reset_index().sort_values(by='NUMERO_COO', ascending=False)
display(cancelados_op)
display(cancelados_op.loc[cancelados_op['FILIAL'] == '135'])

,CAIXA,NOME_OP,FILIAL,NUMERO_COO
321,127461,MAYARA LEITAO BRUNO,999,45
469,132532,RENNAN DOS SANTOS BORGES NASCIMENTO,727,18
583,135466,PEDRO HENRIQUE SANTOS SARAIVA,135,17
275,125059,BENAIA NASCIMENTO PULQUERIO DE ALMEIDA,1078,17
385,130254,LUIZ FELIPE CORREIA DE MELO,106,15
...,...,...,...,...
471,132555,DARCICLEI CAMPOS ALMEIDA,645,1
463,132412,ALINE DE CASTRO ANDRADE,773,1
456,132285,JOSE DE RIBAMAR SILVA NETO,1367,1
455,132281,APARECIDA TAYS PIRES DE SA ALENCAR LIMA,924,1


,CAIXA,NOME_OP,FILIAL,NUMERO_COO
583,135466,PEDRO HENRIQUE SANTOS SARAIVA,135,17
923,80378,LUZINETE ROSA DE PONTES,135,9
339,128560,EDSUSANCLEIDY PEREIRA DA SILVA,135,7
908,76152,DAVI ADONAY GOMES DE SENA,135,3
486,132871,MARIA CLARA DE CARVALHO FERREIRA,135,2
60,110516,WEBER SOARES DE LIMA,135,2


In [28]:
cancelados_pv = df_canc[['FILIAL', 'NUMERO_COO']].drop_duplicates(subset=['FILIAL', 'NUMERO_COO'])
cancelados_pv = cancelados_pv.groupby('FILIAL').count()['NUMERO_COO'].reset_index().sort_values(by='NUMERO_COO', ascending=False)
cancelados_pv.rename(columns={'NUMERO_COO': 'QTD_CANCELADO'}, inplace=True)
display(cancelados_pv)

,FILIAL,QTD_CANCELADO
192,999,91
9,1078,53
6,106,44
91,51,44
179,904,41
...,...,...
117,66,2
60,251,2
70,320,2
187,965,1


In [82]:
cancelados_aut = df_canc[['AUTORIZADOR', 'NOME_AUT', 'FILIAL', 'NUMERO_COO']].drop_duplicates()
cancelados_aut = cancelados_aut.groupby(['AUTORIZADOR', 'NOME_AUT', 'FILIAL']).count()['NUMERO_COO'].reset_index().sort_values(by='NUMERO_COO', ascending=False)
display(cancelados_aut)
display(cancelados_aut.loc[cancelados_aut['FILIAL'] == '135'])

,AUTORIZADOR,NOME_AUT,FILIAL,NUMERO_COO
868,65907,JOAO PEDRO DE SOUSA LIMA,1345,19
730,38140,DANIEL HANDERSON GALINDO SOARES,492,18
41,104029,ESTEVAO AGRIPINO DA SILVA,106,16
314,121675,NATASHA MODESTO DE SOUSA,999,14
129,111047,ANTONIA CRISTIANE OLIVEIRA DA SILVA,999,14
...,...,...,...,...
434,127677,DEISE MISIAK,1204,1
433,127593,ERIKA MARGARETH ROCHA SANTOS RUFINO,1454,1
791,53940,DEUSIMARIA SEVERO DA SILVA,1341,1
561,132849,JOSILDO ALVES DO AMARAL JUNIOR,793,1


,AUTORIZADOR,NOME_AUT,FILIAL,NUMERO_COO
121,110516,WEBER SOARES DE LIMA,135,14
342,123186,MARCOS MAX LEITE DA SILVA,135,13
719,34979,EDNALDA ALVES DE FRANCA MARINHO,135,3
1136,87179,ADRIANO JOSE DA SILVA,135,3
604,13427,AMAURI JOSE DOS SANTOS,135,3
887,68421,GENILSON AUGUSTO DA SILVA,135,2
677,25000,POLICENA TERCEIRA BARBOZA DE ARAUJO,135,1
104,109317,STEPHANIE SOARES MARTINS CARNEIRO,135,1


In [69]:
display(df_canc.columns)

Index(['DATA_INVENTARIO', 'DATA_CPM', 'HORA_AB', 'HORA_FC', 'CHV_FLIAL_PRD',
       'SKU', 'FILIAL', 'ECF', 'CAIXA', 'NOME_OP', 'NUMERO_COO', 'COD_PROD',
       'DIG', 'DESCRICAO', 'QUANT', 'VALOR', 'CANC', 'AUTORIZADOR', 'NOME_AUT',
       'FUNCAO', 'ITEM_CANC', 'MOTIVO', 'TIPO_MOV', 'QTD_FALTA'],
      dtype='object')

In [83]:
df_prd = df_canc[['CAIXA', 
                  'NOME_OP', 
                  'FILIAL', 
                  'NUMERO_COO',
                  'COD_PROD' ,
                  'DESCRICAO',
                  'VALOR',
                  'QTD_FALTA']].drop_duplicates().groupby(['CAIXA', 
                                                           'NOME_OP', 
                                                           'FILIAL',
                                                           'COD_PROD', 
                                                           'DESCRICAO',
                                                           'VALOR',
                                                           'QTD_FALTA']).count()['NUMERO_COO'].reset_index().sort_values(by='NUMERO_COO', ascending=False)
df_prd.rename(columns={'NUMERO_COO': 'QTD_CANCELADO'}, inplace=True)
df_prd.rename(columns={'VALOR': 'VALOR_UNID'}, inplace=True)
df_prd['VALOR TOTAL'] = df_prd['VALOR_UNID'] * df_prd['QTD_CANCELADO']
df_prd['VALOR PERDA'] = df_prd['VALOR_UNID'] * df_prd['QTD_FALTA']
display(df_prd.sort_values(by='QTD_CANCELADO', ascending=False))
display(df_prd.loc[df_prd['FILIAL'] == '135'].sort_values(by='QTD_CANCELADO', ascending=False))

,CAIXA,NOME_OP,FILIAL,COD_PROD,DESCRICAO,VALOR_UNID,QTD_FALTA,QTD_CANCELADO,VALOR TOTAL,VALOR PERDA
1705,134170,LETICIA ALVES MORAES,1124,51456,CERAVE LC FACIAL HID 52ML,88.99,1,3,266.97,88.99
2767,70212904,JESSILENE RAMOS DA SILVA,7102,51412,LACDAY 10000 FCC ALU CPD/8,24.99,1,3,74.97,24.99
949,127461,MAYARA LEITAO BRUNO,999,37231,AGUA MINERAL CRYSTAL SP S/GAS 500ML,4.99,55,2,9.98,274.45
636,123005,STEFANY PEREIRA LIMA,1078,60615,ALCOOL 70% LAVANDA SANARGERM 50ML,15.98,2,2,31.96,31.96
635,123005,STEFANY PEREIRA LIMA,1078,59117,BENEGRIP IMUNO ENERGY EFERV CPD/10,27.00,3,2,54.00,81.00
...,...,...,...,...,...,...,...,...,...,...
3037,89784,FABYOLA MAYARA GOMES DE BARROS,910,57120,"BEPANTOL DERMA PROT LAB FPS50 4,5G",51.99,3,1,51.99,155.97
3038,89784,FABYOLA MAYARA GOMES DE BARROS,910,61713,DES DOVE ORIGINAL AER 250ML,24.49,16,1,24.49,391.84
3039,89784,FABYOLA MAYARA GOMES DE BARROS,910,62527,DORFLEX ENV CPD/10 NV,7.89,2,1,7.89,15.78
3040,89935,ELIZANGELA MARIA OLIVA DE CARVALHO,671,13135,CR ASSAD T XUXINHA INF 45G,14.99,1,1,14.99,14.99


,CAIXA,NOME_OP,FILIAL,COD_PROD,DESCRICAO,VALOR_UNID,QTD_FALTA,QTD_CANCELADO,VALOR TOTAL,VALOR PERDA
1567,132871,MARIA CLARA DE CARVALHO FERREIRA,135,44273,LC HIDRATANTE UREIA 10/ DAUF 150G,29.99,1,1,29.99,29.99
1568,132871,MARIA CLARA DE CARVALHO FERREIRA,135,50150,LACDAY 10000 FCC ALU TAB/60,112.50,1,1,112.50,112.50
2841,76152,DAVI ADONAY GOMES DE SENA,135,76942,DAUF AUTOTESTE COVID-19 ANT NASAL,39.90,1,1,39.90,39.90
2877,80378,LUZINETE ROSA DE PONTES,135,50514,INSUL BASAGLAR KWIKPEN 100UI 3MLC/5,255.42,2,1,255.42,510.84
2878,80378,LUZINETE ROSA DE PONTES,135,50844,REPEL REPELEX LC 200ML C/25%,24.99,1,1,24.99,24.99
2879,80378,LUZINETE ROSA DE PONTES,135,52191,FLORENT 200MG ENV/4,31.00,1,1,31.00,31.00
2880,80378,LUZINETE ROSA DE PONTES,135,56564,BEXAI 35MG CAPS/20,143.00,1,1,143.00,143.00
2881,80378,LUZINETE ROSA DE PONTES,135,61364,ACETILCISTEINA 600MG ENV/16 GN-GEO,27.05,1,1,27.05,27.05
2882,80378,LUZINETE ROSA DE PONTES,135,61712,SAB DOVE ANTIBAC CUIDA/PROTEGE90GX6,22.90,1,1,22.90,22.90
2883,80378,LUZINETE ROSA DE PONTES,135,74890,ATIVDAY VITC TRIPLA ACAO CPD/30 NV,36.99,1,1,36.99,36.99
